In [105]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [107]:
# URL artikel yang akan di-scrape
url = 'https://www.tempo.co/ekonomi/seknas-fitra-makan-bergizi-gratis-belum-dongkrak-pertumbuhan-ekonomi-1533866'

# Mengirim permintaan HTTP ke URL
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Menyimpan hasil scraping di dictionary
hasil = {}

In [ ]:
# Mendapatkan judul berita
judul = soup.find('h1', class_='text-[26px] font-bold leading-[122%] text-neutral-1200')
hasil['judul'] = judul.get_text(strip=True) if judul else 'Tidak ditemukan'

# Mendapatkan sub-judul berita
sub_judul = soup.find('div', class_='font-roboserif leading-[156%] text-neutral-1100')
hasil['sub_judul'] = sub_judul.get_text(strip=True) if sub_judul else 'Tidak ditemukan'

# Mendapatkan isi berita
isi_paragraf = []
isi_berita = soup.find_all('div', id='content-wrapper', class_='max-lg:container xl')

for i in isi_berita:
    paragraf = i.find_all('p')
    for p in paragraf:
        teks = p.get_text(strip=True)
        if teks:  #menambahkan teks bila ada
            isi_paragraf.append(teks)
ringkasan = '\n\n'.join(isi_paragraf)
hasil['isi'] = ringkasan if ringkasan else 'Tidak ditemukan'


# Mendapatkan tanggal publikasi
tanggal_publikasi = soup.find('p', class_='text-neutral-900 text-sm').get_text(strip=True)
if isinstance(tanggal_publikasi, str):
    tanggal, jam = [part.strip() for part in upload.split('|')]
hasil['tanggal'] = tanggal if tanggal else 'Tidak ditemukan'
hasil['jam'] = jam if jam else 'Tidak ditemukan'

# Mendapatkan kategori berita
kategori = soup.find('span', class_='text-sm font-medium text-primary-main')
hasil['kategori'] = kategori.get_text(strip=True) if kategori else 'Tidak ditemukan' 

# Menampilkan hasil dari dictionary (kat = kategori, res = result)
for kat, res in hasil.items():
    print(f"{kat.capitalize()}: {res}\n")

Judul: Seknas Fitra: Makan Bergizi Gratis Belum Dongkrak Pertumbuhan Ekonomi

Sub_judul: Karena memaksakan Makan Bergizi Gratis pemerintah berpeluang mengeliminasi program-program prioritas lain yang tak kalah penting.

Tanggal: 24 Mei 2025

Jam: 17.24 WIB

Kategori: Bisnis

Isi: TEMPO.CO,Jakarta-Sekretariat Nasional Forum Indonesia untuk Transparansi Anggaran (Seknas Fitra) mengkritik programMakan Bergizi Gratis(MBG) yang diproyeksikan menelan biaya hingga Rp 217,8 triliun. Dengan anggaran sebesar itu, program ini dinilai belum mampu mendorong pertumbuhan ekonomi yang stagnan di angka 4,87 persen pada kuartal I 2025."Program MBG yang digadang akan memberikan multiplier effect pada sektor UMKM, petani, peternak dan nelayan justru belum dapat mendongkrak pertumbuhan ekonomi," ujar Sekretaris Jenderal Seknas Fitra, Misbah Hasan, dalam diskusi yang disiarkan secara daring, Jumat, 23 Mei 2025.

Misbah menilai pemerintah perlu legawa mengakui masih banyak persoalan dalam pelaksanaan program

# Tidy up with function

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_tempo(url) -> str:
    # Inisialisasi dictionary hasil
    hasil = {}

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Judul
        judul = soup.find('h1', class_='text-[26px] font-bold leading-[122%] text-neutral-1200')
        hasil['judul'] = judul.get_text(strip=True) if judul else 'Tidak ditemukan'

        # Sub judul
        sub_judul = soup.find('div', class_='font-roboserif leading-[156%] text-neutral-1100')
        hasil['sub_judul'] = sub_judul.get_text(strip=True) if sub_judul else 'Tidak ditemukan'

        # Isi berita
        isi_paragraf = []
        isi_berita = soup.find_all('div', id='content-wrapper', class_='max-lg:container xl')

        for i in isi_berita:
            paragraf = i.find_all('p')
            for p in paragraf:
                teks = p.get_text(strip=True)
                if teks:  #menambahkan teks bila ada
                    isi_paragraf.append(teks)
        ringkasan = '\n\n'.join(isi_paragraf)
        hasil['isi'] = ringkasan if ringkasan else 'Tidak ditemukan'

        # Tanggal & Jam publikasi
        tanggal_publikasi = soup.find('p', class_='text-neutral-900 text-sm')
        if tanggal_publikasi:
            waktu = tanggal_publikasi.get_text(strip=True)
            if '|' in waktu:
                tanggal, jam = [part.strip() for part in waktu.split('|')]
                hasil['tanggal'] = tanggal
                hasil['jam'] = jam
            else:
                hasil['tanggal'] = waktu
                hasil['jam'] = 'Tidak ditemukan'
        else:
            hasil['tanggal'] = 'Tidak ditemukan'
            hasil['jam'] = 'Tidak ditemukan'

        # Kategori
        kategori = soup.find('span', class_='text-sm font-medium text-primary-main')
        hasil['kategori'] = kategori.get_text(strip=True) if kategori else 'Tidak ditemukan'

    except Exception as e:
        print(f"Terjadi kesalahan saat scraping: {e}")
        return None

    # Kembalikan juga sebagai DataFrame
    df = pd.DataFrame([hasil])
    return df

In [144]:
url = 'https://www.tempo.co/ekonomi/potensi-masalah-dari-rencana-pemerintah-ubah-lapas-jadi-perumahan-1533913'
df_hasil = scrape_tempo(url)
df_hasil

,judul,sub_judul,isi,tanggal,jam,kategori
0,Potensi Masalah dari Rencana Pemerintah Ubah L...,Rencana ini berpotensi melanggar hak asasi par...,"TEMPO.CO,Jakarta- Menteri Perumahan dan Kawasa...",24 Mei 2025,21.00 WIB,Bisnis


In [ ]:
# extract file ke csv
df_hasil.to_csv('tempo_artikel.csv', index=False, encoding='utf-8-sig')